In [1]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [2]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [3]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
certificate = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [4]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [5]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        certificate_rating = store.p.find("span", class_ = 'certificate').text if store.p.find("span", class_ = 'certificate') else np.nan
        certificate.append(certificate_rating)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [6]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie_Name": movie_name, "Year_of_Release" : year, "C_Rating": certificate,  "Watch_Time_(min)": time, "Genre": genre , "Movie_Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross_($M)": gross, "Description": description  })

In [7]:
movie_list.head(20)

,Movie_Name,Year_of_Release,C_Rating,Watch_Time_(min),Genre,Movie_Rating,Metascore,Votes,Gross_($M),Description
0,Guardians of the Galaxy Vol. 3,2023,PG-13,150,"Action, Adventure, Comedy",8.3,64,113470,NaN,"Still reeling from the loss of Gamora, Peter Q..."
1,Evil Dead Rise,2023,R,96,Horror,6.8,69,58544,NaN,A twisted tale of two estranged sisters whose ...
2,Dungeons & Dragons: Honor Among Thieves,2023,PG-13,134,"Action, Adventure, Comedy",7.4,72,93610,NaN,A charming thief and a band of unlikely advent...
3,The Covenant,2023,R,123,"Action, Thriller, War",7.6,63,25127,NaN,"During the war in Afghanistan, a local interpr..."
4,A Man Called Otto,2022,PG-13,126,"Comedy, Drama",7.5,51,77037,62.47,Otto is a grump who's given up on life followi...
5,The Super Mario Bros. Movie,2023,PG,92,"Animation, Adventure, Comedy",7.3,46,105011,NaN,The story of The Super Mario Bros. on their jo...
6,Air,2023,R,111,"Drama, Sport",7.5,73,54466,NaN,Follows the history of shoe salesman Sonny Vac...
7,The Mother,2023,R,115,"Action, Thriller",5.5,45,15508,NaN,"While fleeing from dangerous assailants, an as..."
8,The Pope's Exorcist,2023,R,103,"Horror, Thriller",6.1,45,28734,NaN,"Follow Gabriele Amorth, the Vatican's leading ..."
9,Renfield,2023,R,93,"Comedy, Fantasy, Horror",6.4,53,32487,NaN,"Renfield, Dracula's henchman and inmate at the..."


In [9]:
movie_list.to_csv("TEST.csv")